# Bridging CT segmentation and nodule candidate classification
## NoduleAnalysis App

### NoduleAnalysisApp.main()
The next step is to convert the segmentation output into sample tuples. Grouping the data involves finding the dashed outline around what;s flagged by the model, and then taking the segmentation output, which is the voxels flagged by the segmentation model, and coordfinate the center of each lump of flagged voxels as index, row and column.  
#### LunaDataset (val stride=10, and isvallset = true)
First time the LunaDataset is called by running the nodule_analysis.py script, there is no values inside the candidateInfo_list, hence why we extract the values while using cache. 
1. If it's the validation set, then extract every Xth (val_stride) value from the series set, otherwise delete the validation entries and that's your training set. The splitting is at the CT/series_uid level. __getitem__ atribute.  When accessing an it gives batch_size * 48x48 matrices. 
2. For nodule candidatw generation, we iterate over all the series_set to extract their corresponding CT. Each object includes the 121x512x512 CT scan. 
3. <strong>segmentCt</strong> <br>
This line of code runs the segmentation model on the CT scan.
    - <strong>a.</strong> seg_dl<br> 
    Every CT scan has 121 slices the index direction. When accessing an index of the Segmentation dataset, we are given a 7x512x512 CT scan, a posiive mask of dimensions 1x512x512, the series_uid and the slice index.
    -  <strong>b.</strong><br> The segmentation model takes batch_size x 7 context slices x 512 x 512 as input, which means in terms of width and length covers the entire area of the CT scan. The output is batch_size x 1 x 512 x 512. The prediction is a boolean mask
    Note that during training, the last two dimensions of the input if 64 by 64, representing just voxels. 
    -  <strong>c.</strong><br> Prior to this, we've been able to create the boolean mask for all channels accross the CT scan. Here we apply binary erosion which is a mathematical morhology structuring element for shrinking the shapes in an image. 

4. <strong>groupSegmentationOutput</strong> <br>

- a. The label function will take nonzero pixels and mark them as belonging to the same group. The second part of the output is the number of objects found.
- b. Gets the center mass coordinates for each group in the IRC coordinates. This function is intented for center mass calculation, which is why we offset the array by adding 1001 to all pixel density values. 
- c. Here we build our candidate info tuple and append it to the list of detections. 
5. <strong>Nodule Classification</strong> <br>
The classification model resides in the classifications directory. The LunaModel is trained on the LunaData. The LunaData are basically 32x48x48 data that are being classified 0(non-nodule) or 1 (nodule). A sample that is not a nodule has [1, 0] as a label. 
- a. Column 1 of the probability predictions is the predicted probability that should be kept. 
- b. We are zipping the IRC center coordinates, nodule probability, malignancy probability, and also we get the center coordinates in XYZ representation. 
6. If we have the ground truth data which resides in the candidateInfo_dict, compute and pring the confusion matrix and add the results to the total.
- a. The Complete Miss column is basicaly Nodules that were completely missed, wether it they were Benign or Malignant. Anythings that makes it through the predicted nodules, are the ones that survived the nodule classifier. Any non-nodules under that column are basically False Positive (falsely classified as nodules). The number of Non-nodules that were filtered out were the ones that didn't even make it to the malignancy model, as they correctly had been classified as non-nodules.  

In [ ]:
nant']

    if do_mal:
        col_labels = ['', 'Complete Miss', 'Filtered Out', 'Pred. Benign', 'Pred. Malignant']
    else:
        col_laies UID to use.",
        )

        self.cli_args = parser.parse_args(sys_arg
            log.debug(self.cli_args.classification_path)
            cls_model = LunaModel()
            cls_dict = torch.load(self.cli_args.classification_path, map_location=torch.device('mps'))     
  
            cls_model.load_state_dict(cls_dict['model_state'])
            cls_model.eval()
           # cls_model.to(self.device)
        else:
            cls_model = None

        if self.cli_args.malignancy_path:
            malignancy_model = LunaModel()
            malignancy_dict = torch.load(self.cli_args.malignancy_path)
            malignancy_model.load_state_dict(malignancy_dict['model_state'])
            malignancy_model.eval()
            if self.use_cuda:
                malignancy_model.to(self.device)
        else:
            malignancy_model = None


        return seg_model, cls_model, malignancy_model

    def initModelPath(self, type_str):
        local_path = os.path.join(
            'models',
            type_str + '_{}_{}.{}.state'.format('*', '*', 'best'),
        )

        file_list = glob.glob(local_path)
        if not file_list:
            pretrained_path = os.path.join(

                'models',
                type_str + '_{}_{}.{}.state'.format('*', '*', '*'),
            )
            file_list = glob.glob(pretrained_path)
        else:
            pretrained_path = None

        file_list.sort()

        try:
            return file_list[-1]
        except IndexError:
            log.debug([local_path, pretrained_path, file_list])
            raise
    

    
    def initSegmentationDl(self, series_uid):
        seg_ds = Luna2dSegmentationDataset(
                contextSlices_count=3,
                series_uid=series_uid,
                fullCt_bool=True,
            )
        seg_dl = DataLoader(
            seg_ds,
            batch_size=self.cli_args.batch_size ,
            num_workers=self.cli_args.num_workers,
            pin_memory=0,
        )

        return seg_dl
    

    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))
        val_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=True,
        )  #<1>
        val_set = set(
            candidateInfo_tup.series_uid
            for candidateInfo_tup in val_ds.candidateInfo_list
        )
        positive_set = set(
            candidateInfo_tup.series_uid
            for candidateInfo_tup in getCandidateInfoList()
            if candidateInfo_tup.isNodule_bool
        )

        if self.cli_args.series_uid:
            series_set = set(self.cli_args.series_uid.split(','))
        else:
            series_set = set(
                candidateInfo_tup.series_uid
                for candidateInfo_tup in getCandidateInfoList()
            )

        if self.cli_args.include_train:
            train_list = sorted(series_set - val_set)
        else:
            train_list = []
        val_list = sorted(series_set & val_set)
      

        candidateInfo_dict = getCandidateInfoDict()
        series_iter = enumerateWithEstimate(
            val_list + train_list,
            "Series",
        )

        all_confusion = np.zeros((3, 4), dtype=np.int)

        for _, series_uid in series_iter:
            ct = getCt(series_uid) #<2>
            mask_a = self.segmentCt(ct, series_uid) #<3>
            candidateInfo_list = self.groupSegmentationOutput(
                series_uid, ct, mask_a) #<4>
           
            classifications_list = self.classifyCandidates(
                ct, candidateInfo_list) #<5>
            if not self.cli_args.run_validation:
                print(f"found nodule candidates in {series_uid}:")
                for prob, prob_mal, center_xyz, center_irc in classifications_list:
                    if prob > 0.5: 
                        s = f"nodule prob {prob:.3f}, "
                        if self.malignancy_model:
                            s += f"malignancy prob {prob_mal:.3f}, "
                        s += f"center xyz {center_xyz}"
                        print(s)
            if series_uid in candidateInfo_dict: #<6>
                one_confusion = match_and_score(
                    classifications_list, candidateInfo_dict[series_uid]
                )
                print(one_confusion)
                all_confusion += one_confusion
                print_confusion(series_uid, one_confusion, self.malignancy_model is not None) #<a>
                print_confusion("Total", all_confusion, self.malignancy_model is not None)
      





    def initClassificationDl(self, candidateInfo_list):
        cls_ds = LunaDataset(
                sortby_str='series_uid',
                candidateInfo_list=candidateInfo_list,
            )
        cls_dl = DataLoader(
            cls_ds,
            batch_size=self.cli_args.batch_size * (torch.cuda.device_count() if self.use_cuda else 1),
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda,
        )
        return cls_dl
    





    def classifyCandidates(self, ct, candidateInfo_list):
        cls_dl = self.initClassificationDl(candidateInfo_list)
        classifications_list = []
        for batch_ndx, batch_tup in enumerate(cls_dl):
            input_t, _, _, series_list, center_list = batch_tup

            #input_g = input_t.to(self.device)
            with torch.no_grad():
                _, probability_nodule_g = self.cls_model(input_t)
                if self.malignancy_model is not None:
                    _, probability_mal_g = self.malignancy_model(input_t)
                else:
                    probability_mal_g = torch.zeros_like(probability_nodule_g)

            zip_iter = zip(center_list,
                probability_nodule_g[:,1].tolist(),
                probability_mal_g[:,1].tolist())
            for center_irc, prob_nodule, prob_mal in zip_iter:
                center_xyz = irc2xyz(center_irc,
                    direction_a=ct.direction_a,
                    origin_xyz=ct.origin_xyz,
                    vxSize_xyz=ct.vxSize_xyz,
                )
                cls_tup = (prob_nodule, prob_mal, center_xyz, center_irc)
                classifications_list.append(cls_tup) #<b>
        return classifications_list








            
    def groupSegmentationOutput(self, series_uid,  ct, clean_a):
        candidateLabel_a, candidate_count = measurements.label(clean_a) #<a>
        centerIrc_list = measurements.center_of_mass(
            ct.hu_a.clip(-1000, 1000) + 1001,
            labels=candidateLabel_a,
            index=np.arange(1, candidate_count+1),
        )  #<b>

        candidateInfo_list = []
        for i, center_irc in enumerate(centerIrc_list):
            center_xyz = irc2xyz(
                center_irc,
                ct.origin_xyz,
                ct.vxSize_xyz,
                ct.direction_a,
            )
            assert np.all(np.isfinite(center_irc)), repr(['irc', center_irc, i, candidate_count])
            assert np.all(np.isfinite(center_xyz)), repr(['xyz', center_xyz])
            candidateInfo_tup = \
                CandidateInfoTuple(False, False, False, 0.0, series_uid, center_xyz) #<c> 
            candidateInfo_list.append(candidateInfo_tup)
        
        return candidateInfo_list
    


    def segmentCt(self, ct, series_uid):
        with torch.no_grad():
            output_a = np.zeros_like(ct.hu_a, dtype=np.float32)
            seg_dl = self.initSegmentationDl(series_uid)  #  <a>
            for input_t, _, _, slice_ndx_list in seg_dl:
                input_g = input_t.to(self.device)
                prediction_g = self.seg_model(input_g) #<b>
                for i, slice_ndx in enumerate(slice_ndx_list):
                    output_a[slice_ndx] = prediction_g[i].cpu().numpy()

            mask_a = output_a > 0.5
            mask_a = morphology.binary_erosion(mask_a, iterations=1) #<c>
        return mask_a


if __name__ == '__main__':
    NoduleAnalysisApp().main()
